In [2]:
# Modules de base
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd 

# Module relatif à Gurobi
from gurobipy import *

# Module csv
import csv

## Importation des données

In [3]:
# Instance_file : str
Instance_file = "InstanceBordeauxV1.xlsx"

# Chargement des donnees
Employees = pd.read_excel(Instance_file, sheet_name= 'Employees')
Tasks = pd.read_excel(Instance_file, sheet_name= 'Tasks')
Employees_Unavailabilities = pd.read_excel(Instance_file, sheet_name= 'Employees Unavailabilities')
Tasks_Unavailabilities = pd.read_excel(Instance_file, sheet_name= 'Tasks Unavailabilities')

print(Employees)

  EmployeeName   Latitude  Longitude     Skill  Level WorkingStartTime  \
0     Valentin  45.151218  -0.822093  Oenology      2           8:00am   
1        Ambre  45.151218  -0.822093  Oenology      1           8:00am   

  WorkingEndTime  
0         6:00pm  
1         6:00pm  


## Création des dictionnaires de paramètres

Listes des dictionnaires de paramètres à créer:
- LevelEmployee[i] qui à un employé i lui associe son niveau de compétence
- BeginningEmployee[i] qui à un employé i lui associe son début de journée de travail
- EndEmployee[i] qui à un employé i lui associe sa fin de journée de travail
- LevelTask[j] qui à une tâche j lui associe son niveau de difficulté
- BeginningTask[j] qui à une tâche j lui aussi le début du créneau durant lequel on peut l'effectuer
- EndTask[j] qui à une tâche j lui aussi la fin du créneau durant lequel on peut l'effectuer
- DurationTask[j] qui à une tâche j lui associe sa durée
- TasksUnavailabilities[j]
- DistDict[i][j] dictionnaire contenant les distances entre toutes les tâches et les domiciles des employées
- TimeDict[i][j] dictionnaire contenant les temps de trajet associés
- Unavailability[l]: recense les indisponibilités sous forme (employe, debutindispo, duree)

In [84]:
#Paramètres
N = len(Employees["EmployeeName"])
K = len(Tasks["TaskId"])
L = len(Tasks_Unavailabilities["TaskId"])



LevelEmployee = dict()

for i in range (len(Employees["EmployeeName"])):
    a = Employees["EmployeeName"][i]
    b = Employees["Level"][i]
    LevelEmployee[i+1] = b

#print(LevelEmployee)

BeginningEmployee = dict()

for i in range (len(Employees["EmployeeName"])):
    a = Employees["EmployeeName"][i]
    b = Employees["WorkingStartTime"][i][:1]
    bg = b[:2]   #prend les deux premiers caractères
    bd = b[-3:]
    if bg[-1:] == ":":  #cas où l'heure n'a qu'un chiffre
        b = int(b[:1]) + int(bd[:2]) *100/60
        
    if bg[-1:] != ":":
        b = int(b[:2]) + int(bd[-2:]) *100/60

    if Employees["WorkingStartTime"][i][-2:] == 'am':
        BeginningEmployee[a] = b
    if Employees["WorkingStartTime"][i][2:] == "pm":
        BeginningEmployee[a] = b + 12

#print(BeginningEmployee)

EndEmployee = dict()

for i in range (len(Employees["EmployeeName"])):
    a = Employees["EmployeeName"][i]
    b = Employees["WorkingEndTime"][i][:1]
    bg = b[:2]   #prend les deux premiers caractères
    bd = b[-3:]
    if bg[-1:] == ":":  #cas où l'heure n'a qu'un chiffre
        b = int(b[:1]) + int(bd[:2]) *100/60
        
    if bg[-1:] != ":":
        b = int(b[:2]) + int(bd[-2:]) *100/60

    if Employees["WorkingEndTime"][i][-2:] == 'am':
        EndEmployee[a] = b
    if Employees["WorkingEndTime"][i][2:] == "pm":
        EndEmployee[a] = b + 12

#print(EndEmployee)

LevelTask = dict()

for i in range (len(Tasks["TaskId"])):
    a = i+1
    b = Tasks["Level"][i]
    LevelTask[a] = b

#print(LevelTask)

TasksUnavailabilities = dict()

for i in range (len(Tasks_Unavailabilities["TaskId"])):
    a = Tasks_Unavailabilities["TaskId"][i] 
    b = Tasks_Unavailabilities["Start"][i][:5]
    bg = b[:2]   #prend les deux premiers caractères
    bd = b[-3:]
    if bg[-1:] == ":":  #cas où l'heure n'a qu'un chiffre
        b = int(b[:1]) + int(bd[:2]) *100/60
        
    if bg[-1:] != ":":
        b = int(b[:2]) + int(bd[-2:]) *100/60
        
    c = Tasks_Unavailabilities["End"][i][:5]
    cg = c[:2]   #prend les deux premiers caractères
    cd = c[-3:]
    if cg[-1:] == ":":  #cas où l'heure n'a qu'un chiffre
        c = int(c[:1]) + int(cd[:2]) *100/60
        
    if cg[-1:] != ":":
        c = int(c[:2]) + int(cd[-2:]) *100/60

    if Tasks_Unavailabilities["Start"][i][-2:] == 'pm' and b!=12:
        b = b + 12
    if Tasks_Unavailabilities["End"][i][2:] == "pm" and c!=12:
        c = c + 12
    TasksUnavailabilities[a] = [b,c]

#print(TasksUnavailabilities)

Unavailability = dict()

for i in range (len(Employees_Unavailabilities["EmployeeName"])):
    a = Employees_Unavailabilities["EmployeeName"][i] 
    b = Employees_Unavailabilities["Start"][i][:5]
    bg = b[:2]   #prend les deux premiers caractères
    bd = b[-3:]
    if bg[-1:] == ":":  #cas où l'heure n'a qu'un chiffre
        b = int(b[:1]) + int(bd[:2]) *100/60
        
    if bg[-1:] != ":":
        b = int(b[:2]) + int(bd[-2:]) *100/60
        
    c = Employees_Unavailabilities["End"][i][:5]
    cg = c[:2]   #prend les deux premiers caractères
    cd = c[-3:]
    if cg[-1:] == ":":  #cas où l'heure n'a qu'un chiffre
        c = int(c[:1]) + int(cd[:2]) *100/60
        
    if cg[-1:] != ":":
        c = int(c[:2]) + int(cd[-2:]) *100/60
    if Employees_Unavailabilities["Start"][i][-2:] == 'pm'and b!=12:
        b = int(b) + 12
    if Employees_Unavailabilities["End"][i][2:] == "pm" and c!=12:
        c = int(c) + 12 - b
    Unavailability[i] = [a,b,c]

#print(Unavailability)


BeginningTask = dict()

for i in range (K+L+2*N): 
    if i < L:
        a = Employees_Unavailabilities["EmployeeName"][i]
        b = Employees_Unavailabilities["Start"][i][:5]
        c = Employees_Unavailabilities["Start"][i]
    if i >= L and i < L+K:
        a = Tasks["TaskId"][i-L]
        b = Tasks["OpeningTime"][i-L][:5]
        c = Tasks["OpeningTime"][i-L]
    if i >= L+K and i < L+K+N:
        a = Employees["EmployeeName"][i-L-K]
        b = Employees["WorkingStartTime"][i-L-K][:5]
        c = Employees["WorkingStartTime"][i-L-K]
    if i >= L+K+N:
        a = Employees["EmployeeName"][i-L-K-N]
        b = Employees["WorkingEndTime"][i-L-K-N][:5]
        c = Employees["WorkingEndTime"][i-L-K-N]
    bg = b[:2]   #prend les deux premiers caractères
    bd = b[-3:]
    
    if bg[-1:] == ":":  #cas où l'heure n'a qu'un chiffre
        b = int(b[:1]) + int(bd[:2]) *100/60
        
    if bg[-1:] != ":":
        b = int(b[:2]) + int(bd[-2:]) *100/60
        
    if c[-2:] == 'am':
        #BeginningTask[a] = b
        BeginningTask[i+1] = b
    if c[-2:] == "pm":
        #BeginningTask[a] = b + 12
        BeginningTask[i+1] = b + 12

 

#print(BeginningTask)


EndTask = dict()

for i in range (K+L+2*N): 
    if i < L:
        a = Employees_Unavailabilities["EmployeeName"][i]
        b = Employees_Unavailabilities["End"][i][:5]
        c = Employees_Unavailabilities["End"][i]
    if i >= L and i < L+K:
        a = Tasks["TaskId"][i-L]
        b = Tasks["ClosingTime"][i-L][:5]
        c = Tasks["ClosingTime"][i-L]
    if i >= L+K and i < L+K+N:
        a = Employees["EmployeeName"][i-L-K]
        b = Employees["WorkingStartTime"][i-L-K][:5]
        c = Employees["WorkingStartTime"][i-L-K]
    if i >= L+K+N:
        a = Employees["EmployeeName"][i-L-K-N]
        b = Employees["WorkingEndTime"][i-L-K-N][:5]
        c = Employees["WorkingEndTime"][i-L-K-N]
    bg = b[:2]   #prend les deux premiers caractères
    bd = b[-3:]
    
    if bg[-1:] == ":":  #cas où l'heure n'a qu'un chiffre
        b = int(b[:1]) + int(bd[:2]) *100/60
        
    if bg[-1:] != ":":
        b = int(b[:2]) + int(bd[-2:]) *100/60
        
    if c[-2:] == 'am':
        #EndTask[a] = b
        EndTask[i+1] = b
    if c[-2:] == "pm":
        #EndTask[a] = b + 12
        EndTask[i+1] = b + 12

print(EndTask)


DurationTask = dict()

for i in range (K):
    a = L + i + 1
    b = Tasks["TaskDuration"][i] / 60
    DurationTask[a] = b
for i in range (L):
    a = i + 1
    b = Unavailability[i][2] / 60
    DurationTask[a] = b
for i in range (N):
    a = L + K + i + 1
    b = 0
    DurationTask[a] = b
    DurationTask[a+N] = b

#print(DurationTask)

##Création du dictionnaire des distances
#DistDic: associe à (i,j) la distance entre les deux tâches

DistDict = dict()
R = 6371  #rayon de la Terre

for i in range (len(Tasks["TaskId"])):               ## ajout des distances entre tasks
    for k in range (len(Tasks["TaskId"])):
        if k!=i:
            L_i = Tasks["Longitude"][i]
            l_i = Tasks["Latitude"][i]
            L_k = Tasks["Longitude"][k]
            l_k = Tasks["Latitude"][k]
            A = np.sin((l_i-l_k)*np.pi/(2*180))**2
            B = (np.sin((L_i-L_k)*np.pi/(2*180))**2)*np.cos(l_i*np.pi/180)*np.cos(l_k*np.pi/180)
            d = 2*R*np.sqrt(A+B)
            DistDict[(L+i+1,L+k+1)] = d
            DistDict[(L+k+1,L+i+1)] = d

for i in range (K):             #### ajout des disatnces tasks - domiciles des employés
    for k in range (N):
        L_i = Tasks["Longitude"][i]
        l_i = Tasks["Latitude"][i]
        L_E = Employees["Longitude"][k]
        l_E = Employees["Latitude"][k]
        A = np.sin((l_i-l_E)*np.pi/(2*180))**2
        B = (np.sin((L_i-L_E)*np.pi/(2*180))**2)*np.cos(l_i*np.pi/180)*np.cos(l_E*np.pi/180)
        d = 2*R*np.sqrt(A+B)
        DistDict[(L+i+1, L+K+k+1) ] = d 
        DistDict[(L+K+k+1, L+i+1)] = d 
        DistDict[(L+i+1, L+K+N+k+1)] = d
        DistDict[(L+K+N+k+1,L+i+1)] = d       

for i in range (len(Tasks["TaskId"])):      #ona joute les distances entre les tâches et les indispos
    for k in range (len(Employees_Unavailabilities["EmployeeName"])):
        L_i = Tasks["Longitude"][i]
        l_i = Tasks["Latitude"][i]
        L_E = Employees_Unavailabilities["Longitude"][k]
        l_E = Employees_Unavailabilities["Latitude"][k]
        A = np.sin((l_i-l_E)*np.pi/(2*180))**2
        B = (np.sin((L_i-L_E)*np.pi/(2*180))**2)*np.cos(l_i*np.pi/180)*np.cos(l_E*np.pi/180)
        d = 2*R*np.sqrt(A+B)
        DistDict[(L+i+1, k+1)] = d #distance entre indispo k et tâche i
        DistDict[(k+1, L+i+1)] = d    
        
for i in range (len(Employees_Unavailabilities["EmployeeName"])):      #ona joute les distances entre les indispos
    for k in range (len(Employees_Unavailabilities["EmployeeName"])):
        L_i = Employees_Unavailabilities["Longitude"][i]
        l_i = Employees_Unavailabilities["Latitude"][i]
        L_E = Employees_Unavailabilities["Longitude"][k]
        l_E = Employees_Unavailabilities["Latitude"][k]
        A = np.sin((l_i-l_E)*np.pi/(2*180))**2
        B = (np.sin((L_i-L_E)*np.pi/(2*180))**2)*np.cos(l_i*np.pi/180)*np.cos(l_E*np.pi/180)
        d = 2*R*np.sqrt(A+B)
        DistDict[(i+1, k+1)] = d #distance entre indispo k et indispo i
        DistDict[(k+1, i+1)] = d    
        
for i in range (len(Employees_Unavailabilities["EmployeeName"])):      #ona joute les distances entre les indispos et les domiciles
    for k in range (len(Employees["EmployeeName"])):
        L_i = Employees_Unavailabilities["Longitude"][i]
        l_i = Employees_Unavailabilities["Latitude"][i]
        L_E = Employees["Longitude"][k]
        l_E = Employees["Latitude"][k]
        A = np.sin((l_i-l_E)*np.pi/(2*180))**2
        B = (np.sin((L_i-L_E)*np.pi/(2*180))**2)*np.cos(l_i*np.pi/180)*np.cos(l_E*np.pi/180)
        d = 2*R*np.sqrt(A+B)
        DistDict[(L+K+k+1, i+1)] = d #distance entre indispo i et employe k
        DistDict[(i+1, L+K+k+1)] = d
        DistDict[(L+K+N+k+1, i+1)] = d 
        DistDict[(i+1, L+K+N+k+1)] = d   

#print(DistDict)

##Création dictionnaire des temps de trajet

# TimeDict : associe à un trajet (i,j) le temps de trajet en h


TimeDict = dict()

for element in DistDict.items():
    t = element[1] / 50
    TimeDict[element[0]] = t

print(TimeDict)



{1: 18.0, 2: 18.0, 3: 18.0, 4: 18.0, 5: 18.0, 6: 18.0, 7: 18.0, 8: 18.0, 9: 18.0, 10: 18.0, 11: 8.0, 12: 8.0, 13: 18.0, 14: 18.0}
{(1, 2): 1.0217165126372043, (2, 1): 1.0217165126372043, (1, 3): 1.4614402548607615, (3, 1): 1.4614402548607615, (1, 4): 1.7287379164063463, (4, 1): 1.7287379164063463, (1, 5): 1.2170045168101324, (5, 1): 1.2170045168101324, (1, 6): 1.5802896694770947, (6, 1): 1.5802896694770947, (1, 7): 2.1298880486390215, (7, 1): 2.1298880486390215, (1, 8): 1.2926838792504103, (8, 1): 1.2926838792504103, (1, 9): 1.90418311153199, (9, 1): 1.90418311153199, (1, 10): 1.3973216507616857, (10, 1): 1.3973216507616857, (2, 3): 0.4397289075591224, (3, 2): 0.4397289075591224, (2, 4): 0.7089972454352003, (4, 2): 0.7089972454352003, (2, 5): 0.19541368070280252, (5, 2): 0.19541368070280252, (2, 6): 0.559462489108554, (6, 2): 0.559462489108554, (2, 7): 1.109249846470811, (7, 2): 1.109249846470811, (2, 8): 0.3361191598312767, (8, 2): 0.3361191598312767, (2, 9): 0.8878647675886318, (9, 2

## Modules

In [77]:
# Modules de base
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.ticker import MultipleLocator, FormatStrFormatter

# Module relatif à Gurobi
from gurobipy import *

## Données

In [78]:
# number_of_employees : int
N=len(Employees)

# number_of_tasks : int
K=len(Tasks)

# number_of_unavailabilities : int
L=len(Unavailability)



## Variables

In [79]:
# -- Initialisation du modèle --
# m : Model
m = Model("ProjetST7")

# -- Ajout des variables  --
# X: dict[(int, int) : Var]
X = {(i,j) : m.addVar(vtype = GRB.BINARY, name = f'employeetask{i}_{j}') for i in range(1, N+1) for j in range(1,K+L+2*N+1)}

# T : dict[int : Var]
T = {(j) : m.addVar(vtype = GRB.CONTINUOUS, name=f'begintask_{j}') for j in range(1, K+L+2*N+1)}

# Y: dict[(int, int) : Var]
Y = {(i,j) : m.addVar(vtype = GRB.BINARY, name = f'successivetask{i}_{j}') for i in range(1, K+L+2*N+1) for j in range(1,K+L+2*N+1)}

## Contraintes

- Un employé pour une tâche

In [80]:
# OneEmployeePerTaskConstrDict : dict[int : Constr]
OneEmployeePerTaskConstrDict = {j : m.addConstr(quicksum([X[(i, j)] for i in range(1, N + 1)]) == 1, name = f'one_employee_per_task{j}') for j in range(1, L+K+2*N + 1)}

- Niveau de l'employé suffisant pour la tâche

In [81]:
# MinimumLevelConstrDict : dict[(int,int) : Constr]
MinimumLevelConstrDict = {(i,j) : m.addConstr(LevelTask[j]*X[(i,j)] <= LevelEmployee[i]  , name = f'level_{i}_{j}') for i in range(1, N + 1) for j in range(L+1,L+K) }

- Début et fin de chaque tâche

In [82]:
# StartConstrDict : dict[(int,int) : Constr]
StartConstrDict = {j : m.addConstr(T[j] >= BeginningTask[j], name = f'start_{j}') for j in range(1,K+L+2*N+1) }
EndConstrDict = {j : m.addConstr(T[j]+DurationTask[j]<=EndTask[j], name = f'end_{j}') for j in range(1,K+L+2*N+1) }

- Délai entre 2 tâches successives

In [109]:
M=10000
#SuccessiveTasksGapConstrDict = {(i,j) : m.addConstr(Y[(i,j)]*(DurationTask[i]+TimeDict[(i,j)])+(1-Y[(i,j)])*(-M)<=T[j]-T[i] , name = f'successivetaksgap_{i}_{j}') for i in range(1, K+L+2*N + 1) for j in range(1,K+L+2*N+1) if i!=j }
SuccessiveTasksGapConstrDict = {(i,j) : m.addConstr(Y[(i,j)]*(DurationTask[i]+TimeDict[(i,j)])+(1-Y[(i,j)])*(-M)-(T[j]-T[i]) <= 0 , name = f'successivetaksgap_{i}_{j}') for i in range(1, K+L+2*N+1) for j in range(1,K+L+2*N+1) if (i,j)  in DistDict.keys() }


- Une seule tâche succède à une autre

In [110]:
OneTaskAfterAnother1ConstrDict = {j : m.addConstr(quicksum(Y[(k,j)] for k in range(1,K+L+2*N))==1, name = f'OneTaskAfterAnother1_{j}') for j in range(2,K+L+2*N+1) }
OneTaskAfterAnother2ConstrDict = {j : m.addConstr(quicksum(Y[(j,k)] for k in range(2,K+L+2*N+1))==1, name = f'OneTaskAfterAnother2_{j}') for j in range(1,K+L+2*N) }
OneTaskAfterAnother3ConstrDict = {j : m.addConstr(quicksum(Y[(j,k)] for k in range(2,K+L+2*N+1))==quicksum(Y[(k,j)] for k in range(1,K+L+2*N)), name = f'OneTaskAfterAnother3_{j}') for j in range(2,K+L+2*N) }


- Si deux tâches sont successives il existe k tel que ces tâches sont effectuées par le même employé

In [111]:
SuccessiveTasksOneEmployee1ConstrDict = {(i,j,k) : m.addConstr(1-Y[(i,j)]>=X[(k,i)]-X[(k,j)] , name = f'successivetaksoneemployee1_{i}_{j}_{k}') for i in range(L+1, K+L+2*N + 1) for j in range(L+1,K+L+2*N+1) for k in range(1,N+1) }
SuccessiveTasksOneEmployee2ConstrDict = {(i,j,k) : m.addConstr(1-Y[(i,j)]>=-(X[(k,i)]-X[(k,j)]) , name = f'successivetaksoneemployee2_{i}_{j}_{k}') for i in range(L+1, K+L+2*N + 1) for j in range(L+1,K+L+2*N+1) for k in range(1,N+1) }
#SuccessiveTasksOneEmployee3ConstrDict = {(i,j,k) : m.addConstr(1-Y[(i,j)]>=-(X[(k,i)]-X[(k,j)]) , name = f'successivetaksoneemployee3_{i}_{j}_{k}') for i in range(1, L+1) for j in range(1,L+1) for k in range(1,N+1) }

## Fonction Objectif

In [112]:
# -- Ajout de la fonction objectif --
m.setObjective(quicksum(Y[(i,j)]*TimeDict[(i,j)] for i in range (1,K+L+2*N+1) for j in range (1,K+L+2*N+1) if (i,j)  in DistDict.keys() ), GRB.MINIMIZE)

In [119]:
# -- Choix d'un paramétrage d'affichage minimaliste --
m.params.outputflag = 0 # mode muet

# -- Mise à jour du modèle  --
m.update()

print(m.status)
# -- Résolution --
m.optimize()



1


GurobiError: Model too large for size-limited license; visit https://www.gurobi.com/free-trial for a full license

In [114]:
print(m.ObjVal)

AttributeError: Unable to retrieve attribute 'ObjVal'

In [115]:
print(X)

{(1, 1): <gurobi.Var employeetask1_1>, (1, 2): <gurobi.Var employeetask1_2>, (1, 3): <gurobi.Var employeetask1_3>, (1, 4): <gurobi.Var employeetask1_4>, (1, 5): <gurobi.Var employeetask1_5>, (1, 6): <gurobi.Var employeetask1_6>, (1, 7): <gurobi.Var employeetask1_7>, (1, 8): <gurobi.Var employeetask1_8>, (1, 9): <gurobi.Var employeetask1_9>, (1, 10): <gurobi.Var employeetask1_10>, (1, 11): <gurobi.Var employeetask1_11>, (1, 12): <gurobi.Var employeetask1_12>, (1, 13): <gurobi.Var employeetask1_13>, (1, 14): <gurobi.Var employeetask1_14>, (2, 1): <gurobi.Var employeetask2_1>, (2, 2): <gurobi.Var employeetask2_2>, (2, 3): <gurobi.Var employeetask2_3>, (2, 4): <gurobi.Var employeetask2_4>, (2, 5): <gurobi.Var employeetask2_5>, (2, 6): <gurobi.Var employeetask2_6>, (2, 7): <gurobi.Var employeetask2_7>, (2, 8): <gurobi.Var employeetask2_8>, (2, 9): <gurobi.Var employeetask2_9>, (2, 10): <gurobi.Var employeetask2_10>, (2, 11): <gurobi.Var employeetask2_11>, (2, 12): <gurobi.Var employeetask2_

In [116]:
print(Y)

{(1, 1): <gurobi.Var successivetask1_1>, (1, 2): <gurobi.Var successivetask1_2>, (1, 3): <gurobi.Var successivetask1_3>, (1, 4): <gurobi.Var successivetask1_4>, (1, 5): <gurobi.Var successivetask1_5>, (1, 6): <gurobi.Var successivetask1_6>, (1, 7): <gurobi.Var successivetask1_7>, (1, 8): <gurobi.Var successivetask1_8>, (1, 9): <gurobi.Var successivetask1_9>, (1, 10): <gurobi.Var successivetask1_10>, (1, 11): <gurobi.Var successivetask1_11>, (1, 12): <gurobi.Var successivetask1_12>, (1, 13): <gurobi.Var successivetask1_13>, (1, 14): <gurobi.Var successivetask1_14>, (2, 1): <gurobi.Var successivetask2_1>, (2, 2): <gurobi.Var successivetask2_2>, (2, 3): <gurobi.Var successivetask2_3>, (2, 4): <gurobi.Var successivetask2_4>, (2, 5): <gurobi.Var successivetask2_5>, (2, 6): <gurobi.Var successivetask2_6>, (2, 7): <gurobi.Var successivetask2_7>, (2, 8): <gurobi.Var successivetask2_8>, (2, 9): <gurobi.Var successivetask2_9>, (2, 10): <gurobi.Var successivetask2_10>, (2, 11): <gurobi.Var succes

In [117]:
print(T)

{1: <gurobi.Var begintask_1>, 2: <gurobi.Var begintask_2>, 3: <gurobi.Var begintask_3>, 4: <gurobi.Var begintask_4>, 5: <gurobi.Var begintask_5>, 6: <gurobi.Var begintask_6>, 7: <gurobi.Var begintask_7>, 8: <gurobi.Var begintask_8>, 9: <gurobi.Var begintask_9>, 10: <gurobi.Var begintask_10>, 11: <gurobi.Var begintask_11>, 12: <gurobi.Var begintask_12>, 13: <gurobi.Var begintask_13>, 14: <gurobi.Var begintask_14>}
